In [1]:
import re
import pandas as pd
from toolz import *
from toolz.curried import *

In [2]:
def preprocess_micro(micro):    
    """
    preprocess micro data properties.csv
    """
    def _floorRange(row, thresFloor = 41):
        """
        some floorRanges are too rare to be used as categorical variables
        Hence merging floorRage above the "thresFloor" into one.
        """
        floorRange = row["floorRange"]    
        if floorRange != "-":
            floorMin = int(re.search("(\d+)-", floorRange).groups()[0])
            if floorMin >= thresFloor:
                row["floorRange"] = f"{thresFloor}-99"    
        return row

    def _tenure(row):
        """
        some tensures are too rare to be used as categorical variables
        Hence simply making tenure feature to 1 if freehold else 0.
        """
        tenure = row["tenure"]    
        if tenure == "Freehold" : row["tenure"] = 1
        else                    : row["tenure"] = 0        
        return row
        
    micro = (micro
             .apply(_floorRange, axis = 1)
             .apply(_tenure, axis = 1))
    
    micro["date"] = pd.to_datetime(micro["date"])
    
    return micro

In [3]:
def preprocess_macros(cpi, interest, rentIndex, vacant):
    """
    four files are considered as macro-related files :
        * cpi.csv
        * interest.csv
        * rentIndex.csv
        * vacant.csv
    """
    
    def yq2ym(df):
        """
        convert year-quater in string format to monthly period
        """
        
        # repeat the first row for desired interpolation result
        df = pd.concat([df.head(0), df])
        df.at[0, "date"] = "2023 1Q "
            
        df["date"] = (pd.to_datetime((df["date"]
                                      .str
                                      .replace(r"(\d+) (\d)Q ", r"\1-Q\2")))
                      .dt
                      .to_period('M'))        
        df = df.set_index("date").resample("M", convention = "end").interpolate("linear")        
        return df    

    def ym2ym(df):
        """
         convert year-month in string format to monthly period
        """
        df["date"] = (pd.to_datetime(df["date"], format = "%Y %b ")
                      .dt
                      .to_period('M'))
        return df
    
    def mergeDfs(dfs, on):
        """
        join dataframes into one
        """
        df = reduce(partial(pd.merge, on = on, how = "inner"), dfs)
        return df
    
    # apply appropriate date conversion function for each dataframe
    dfs = map(apply,
              [ym2ym, ym2ym, yq2ym, yq2ym],
              [cpi, interest, rentIndex, vacant])    
    
    # join all dataframs into one
    df = mergeDfs(dfs, on = "date")   
    # convert peroid[M] to datetime
    df["date"] = df["date"].dt.to_timestamp()    
    # shift month + 3 
    df["date"] = df["date"] + pd.DateOffset(months=+3)
    
    return df 

In [ ]:
trainPath   = "./raw/train.csv"
testPath    = "./raw/test.csv" 
microPath   = "./raw/properties.csv"
macrosPaths = ["./raw/cpi.csv", "./raw/interest.csv", "./raw/rentIndex.csv", "./raw/vacant.csv"]
geoPath     = "./raw/geo_attributes.csv"

train_keys = pd.read_csv(trainPath)
test_keys  = pd.read_csv(testPath)
keys       = pd.concat([train_keys.assign(train = 1),
                        test_keys].assign(train = 0).assign(price = pd.NA))    
keys       = keys.rename(columns = {"contractDate" : "date"})

_micro = pd.read_csv(microPath)
micro  = pd.merge(_micro, keys, on = "property_key")    
macros = compose(list,
                 map(lambda df : df.rename(columns = {"Data Series" : "date"})),
                 map(pd.read_csv))(macrosPaths)
geo    = pd.read_csv(geoPath)

micro = preprocess_micro(micro)
macro = preprocess_macros(*macros)

_merged = pd.merge(micro, macro, on = "date", how = "left")
merged  = pd.merge(_merged, geo, on = ["street", "project", "district"], how = "left")
merged  = merged.fillna(merged.mean())

merged_train = merged[merged["train"] == 1].drop("train", axis=1)
merged_test  = merged[merged["train"] == 0].drop("train", axis=1)

merged_train.to_csv("./processed/merged_train.csv")
merged_test .to_csv("./processed/merged_test.csv")    